In [0]:
# Create Spark Session
from pyspark.sql import SparkSession

In [0]:
spark = SparkSession.builder.appName('lr_example').getOrCreate()

In [0]:
from pyspark.ml.regression import LinearRegression

In [0]:
# lOAD THE data
df = spark.read.csv('/FileStore/tables/Ecommerce_Customers.csv', header=True, inferSchema=True)

In [0]:
df.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [0]:
for item in df.head(2)[1]:
  print(item)

hduke@hotmail.com
4547 Archer CommonDiazchester, CA 06566-8576
DarkGreen
31.92627202636016
11.109460728682564
37.268958868297744
2.66403418213262
392.2049334443264


In [0]:
# In order to create a model we have to import Vector & Vector Assembler
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [0]:
df.columns

Out[18]: ['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [0]:
# Making features into vector columns
assembler = VectorAssembler(inputCols=[ 'Avg Session Length','Time on App','Time on Website','Length of Membership',],
                            outputCol='features')

In [0]:
# Transform the data
output = assembler.transform(df)

In [0]:
output.head(1)

Out[25]: [Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))]

In [0]:
final_data = output.select(['features', 'Yearly Amount Spent'])

In [0]:
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [0]:
train_data, test_data = final_data.randomSplit([0.7, 0.3])

In [0]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                344|
|   mean|  501.6120149741153|
| stddev|  80.03756544139564|
|    min|   266.086340948469|
|    max|  765.5184619388373|
+-------+-------------------+



In [0]:
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                156|
|   mean|  494.2467049884598|
| stddev|  77.71088142271635|
|    min| 256.67058229005585|
|    max|  689.7876041747194|
+-------+-------------------+



In [0]:
# Instantiate Model
lr = LinearRegression(labelCol='Yearly Amount Spent')

In [0]:
# Fit model
lr_model = lr.fit(train_data)

In [0]:
# Evaluate model
test_result = lr_model.evaluate(test_data)

In [0]:
test_result.residuals.show() # Residuals are difference between predicted value and actual labels on test data

+-------------------+
|          residuals|
+-------------------+
| 10.878037137645606|
|  0.358327910136893|
|   10.4157948447222|
|-12.919972722750288|
|-21.808049082883485|
|  -1.13171559614932|
| 2.9093027618479255|
| -4.406282243555324|
| 2.6510227863292926|
| -5.375341904859965|
| -8.863333684616862|
|  3.473813116522308|
| -18.23342066477744|
|-14.748322899406048|
|   16.8210794658267|
|  -4.83212731000782|
|  6.397590585201158|
|-0.8029804794017537|
|  2.516414128515464|
| -4.768516173000592|
+-------------------+
only showing top 20 rows



In [0]:
test_result.rootMeanSquaredError # Difference between test value/true value and predicted value

Out[39]: 9.446524334878365

In [0]:
test_result.r2

Out[40]: 0.9851278647703917

In [0]:
final_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                500|
|   mean|  499.3140382585909|
| stddev|   79.3147815497068|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [0]:
unlabeled_data = test_data.select('features')

In [0]:
# Predicting for unlabeled features 
predict_unlabeld = lr_model.transform(unlabeled_data)

In [0]:
predict_unlabeld.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[29.5324289670579...| 397.7623139349819|
|[30.5743636841713...|441.70608584792876|
|[30.7377203726281...| 451.3649473515077|
|[31.0662181616375...|461.85326593042464|
|[31.1239743499119...|508.75510292264926|
|[31.2606468698795...| 422.4583468531007|
|[31.3091926408918...| 429.8114150780857|
|[31.4252268808548...| 535.1730008983172|
|[31.4459724827577...| 482.2259421487993|
|[31.5257524169682...|449.34096871474185|
|[31.5261978982398...|417.95785987695467|
|[31.5316044825729...|433.04179261284025|
|[31.5702008293202...| 564.1789128061823|
|[31.5741380228732...| 559.1575950599929|
|[31.6098395733896...|427.72447018528146|
|[31.6253601348306...|  381.169028066932|
|[31.6548096756927...|468.86583314234736|
|[31.6610498227460...| 417.1613340593026|
|[31.7366356860502...|494.41703212701645|
|[31.7656188210424...| 501.3225978086077|
+--------------------+------------